In [9]:
import libtorrent as lt

ses = lt.session()
ses.listen_on(6881, 6891)
downloads = []

/tmp/ipykernel_123501/2631723446.py:4: DeprecationWarning: listen_on() is deprecated
  ses.listen_on(6881, 6891)


In [10]:
import os
os.makedirs("/home/nghiapd/medical_dataset/zip_data", exist_ok=True)

In [11]:
torrent_file_path = "/home/nghiapd/medical_dataset/.torrent/MICCAI_BraTS_2018_Data_Validation.zip-a5912da845c7d7bec9bd0880c17ddda789ba35d5.torrent"

params = {
    "save_path": "/home/nghiapd/medical_dataset/zip_data",
    "ti": lt.torrent_info(torrent_file_path),
}
downloads.append(ses.add_torrent(params))

In [12]:
import time
from IPython.display import display
import ipywidgets as widgets

state_str = [
    "queued",
    "checking",
    "downloading metadata",
    "downloading",
    "finished",
    "seeding",
    "allocating",
    "checking fastresume",
]

layout = widgets.Layout(width="auto")
style = {"description_width": "initial"}
download_bars = [
    widgets.FloatSlider(
        step=0.01, disabled=True, layout=layout, style=style
    )
    for _ in downloads
]

display(*download_bars)

while downloads:
    next_shift = 0
    for index, download in enumerate(downloads[:]):
        bar = download_bars[index + next_shift]
        if not download.is_seed():
            s = download.status()

            bar.description = " ".join(
                [
                    download.name(),
                    ":",
                    "%.2f" % (s.progress * 100),
                    "%",
                    "down",
                    "%.1f" % (s.download_rate / 1000),
                    "kB/s",
                    "up",
                    "%.1f" % (s.upload_rate / 1000),
                    "kB/s",
                    "peers",
                    str(s.num_peers),
                    "seeds",
                    str(s.num_seeds),
                    state_str[s.state],
                ]
            )
            bar.value = s.progress * 100
        else:
            next_shift -= 1
            ses.remove_torrent(download)
            downloads.remove(download)
            bar.close() # Seems to be not working in Colab (see https://github.com/googlecolab/colabtools/issues/726#issue-486731758)
            download_bars.remove(bar)
            print("complete")
    time.sleep(1)

FloatSlider(value=0.0, disabled=True, layout=Layout(width='auto'), step=0.01, style=SliderStyle(description_wi…

/tmp/ipykernel_123501/2086606456.py:31: DeprecationWarning: is_seed() is deprecated
  if not download.is_seed():
/tmp/ipykernel_123501/2086606456.py:36: DeprecationWarning: name() is deprecated
  download.name(),


complete


In [ ]:
import os, zipfile

dir_name = '/home/nghiapd/medical_dataset/zip_data/'
extension = ".zip"

os.chdir(dir_name) # change directory from working dir to dir with files

for item in os.listdir(dir_name): # loop through items in dir
    if item.endswith(extension): # check for ".zip" extension
        file_name = os.path.abspath(item) # get full path of files
        zip_ref = zipfile.ZipFile(file_name) # create zipfile object
        zip_ref.extractall(dir_name) # extract file to dir
        zip_ref.close() # close file
        os.remove(file_name) # delete zipped file